In [ ]:
%env WORKDIR=/tmp/vault
%env ROLE_NAME=csi

Importamos la dirección, localización del fichero de la CA de Vault así como el Vault token usando [python-dotenv](https://pypi.org/project/python-dotenv/). De esta forma evitamos filtrar el root token de Vault

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("/tmp/vault/config.env")

VAULT_TOKEN = os.getenv('VAULT_TOKEN')
VAULT_ADDR = os.getenv('VAULT_ADDR')
VAULT_CACERT = os.getenv('VAULT_CACERT')

Creamos una service account que será usada por un POD en el namespace test

In [ ]:
! kubectl create ns csi

In [ ]:
! kubectl create sa $ROLE_NAME -n csi

Extendemos el role creado previamente para permitir el acceso de la SA que se acaba de crear

## Instalamos el CSI driver

In [ ]:
! helm repo add secrets-store-csi-driver https://kubernetes-sigs.github.io/secrets-store-csi-driver/charts

In [ ]:
! helm install csi secrets-store-csi-driver/secrets-store-csi-driver --set syncSecret.enabled=true

In [ ]:
! kubectl get pods

## Instalamos un SecretProviderClass que apunta a Vault

In [ ]:
%%bash
cat > $WORKDIR/spc-vault-database.yaml <<EOF
apiVersion: secrets-store.csi.x-k8s.io/v1
kind: SecretProviderClass
metadata:
  name: vault-database
  namespace: csi
spec:
  provider: vault
  parameters:
    vaultAddress: "https://vault.vault.svc.cluster.local:8200"
    vaultSkipTLSVerify: "true"
    roleName: "role"
    objects: |
      - objectName: "db-username"
        secretPath: "database/creds/readonly"
        secretKey: "username"
      - objectName: "db-password"
        secretPath: "database/creds/readonly"
        secretKey: "password"
EOF


In [ ]:
! kubectl apply --filename $WORKDIR/spc-vault-database.yaml

In [ ]:
! kubectl describe SecretProviderClass vault-database -n csi

Creamos un POD que monte el secreto

In [ ]:
%%bash
cat > $WORKDIR/csi-pod.yaml <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: webapp
  namespace: csi
spec:
  serviceAccountName: $ROLE_NAME
  containers:
  - image: nginx
    name: csi-pod
    volumeMounts:
    - name: secrets-csi
      mountPath: "/mnt/secrets-store"
      readOnly: true
  volumes:
    - name: secrets-csi
      csi:
        driver: secrets-store.csi.k8s.io
        readOnly: true
        volumeAttributes:
          secretProviderClass: "vault-database"
EOF

In [ ]:
! kubectl apply --filename $WORKDIR/csi-pod.yaml

In [ ]:
! kubectl exec webapp -i -t -n csi -- cat /mnt/secrets-store/db-username 
! echo "\n"
! kubectl exec webapp -i -t -n csi -- cat /mnt/secrets-store/db-password 

A diferencia del Vault agent o VSO, los secretos dinámicos no se refrescan en runtime

In [ ]:
%%bash
sleep 600
kubectl exec webapp -i -t -n csi -- cat /mnt/secrets-store/db-username 
echo "\n"
kubectl exec webapp -i -t -n csi -- cat /mnt/secrets-store/db-password 